In [1]:
# !pip install ragas
# !pip install transformers
# !pip3 install torch torchvision torchaudio
# ! pip install pydantic==2.7.3
# !C:\Users\onyxs\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install langchain_core --upgrade
# !pip install -U pydantic pydantic_core 

### python 3.12 can run this you have to be in 3.11
# !C:\Users\onyxs\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade ipywidgets jupyter 
# !C:\Users\onyxs\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip
# !C:\Users\onyxs\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade ragas
# !C:\Users\onyxs\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade transformers
#!C:\Users\onyxs\AppData\Local\Programs\Python\Python311\python.exe -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# !C:\Users\onyxs\AppData\Local\Programs\Python\Python311\python.exe -m pip install TensorFlow Flax 
# !C:\Users\onyxs\AppData\Local\Programs\Python\Python311\python.exe -m pip install fuzzywuzzy python-Levenshtein
# !C:\Users\onyxs\AppData\Local\Programs\Python\Python311\python.exe -m pip install scikit-learn nltk
!C:\Users\onyxs\AppData\Local\Programs\Python\Python311\python.exe -m pip install matplotlib 

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ------------------------ --------------- 5.0/8.0 MB 30.2 MB/s eta 0:00:01
   ---------------------------------------- 8.0/8.0 MB 31.1 MB/s eta 0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 41.3 MB/s eta 0:00:00


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
from ragas.dataset_schema import SingleTurnSample 
from ragas.metrics import FaithfulnesswithHHEM

In [10]:
# from ragas.dataset_schema import SingleTurnSample 
# from ragas.metrics import FaithfulnesswithHHEM
# from transformers import AutoModelForSeq2SeqLM

# my_device = "cuda:0"
# my_batch_size = 10

# sample = SingleTurnSample(
#         user_input="When was the first super bowl?",
#         response="The first superbowl was held on Jan 15, 1967",
#         retrieved_contexts=[
#             "The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles."
#         ]
#     )

# my_device = "cuda:0" if torch.cuda.is_available() else "cpu"
# model_name = "google/flan-t5-xl"  # Or a smaller model for testing
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(my_device)

# scorer = FaithfulnesswithHHEM(llm=model, device=my_device)

# scorer.single_turn_score(sample)

In [9]:
# import asyncio
# from ragas.dataset_schema import SingleTurnSample
# from ragas.metrics import FaithfulnesswithHHEM
# from transformers import AutoModelForSeq2SeqLM
# import torch

# async def evaluate_faithfulness():  # Define an async function
#     my_device = "cuda:0" if torch.cuda.is_available() else "cpu"
#     model_name = "google/flan-t5-xl"
#     model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(my_device)

#     scorer = FaithfulnesswithHHEM(llm=model, device=my_device)

#     sample = SingleTurnSample(
#         user_input="When was the first super bowl?",
#         response="The first superbowl was held on Jan 15, 1967",
#         retrieved_contexts=[
#             "The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles."
#         ]
#     )

#     result = await scorer.single_turn_score(sample)  # Await the result
#     print(result)

# # Run the async function
# asyncio.run(evaluate_faithfulness()) # Correct way to use it

In [8]:
from fuzzywuzzy import fuzz

def calculate_fuzzy_similarity(response, context):
    return fuzz.ratio(response, context)  # Or use other fuzzywuzzy functions

user_input = "When was the first super bowl?"
response = "The first superbowl was held on Jan 15, 1967."
retrieved_context = "The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles."

similarity_score = calculate_fuzzy_similarity(response, retrieved_context)
print(f"Fuzzy Similarity Score: {similarity_score}")


# Example of how to use this in a loop with multiple contexts:
contexts = [
    "The First AFL–NFL World Championship Game was on January 15, 1967.",
    "Super Bowl I was played in 1967.",
    "This is completely unrelated."
]

for context in contexts:
    similarity = calculate_fuzzy_similarity(response, context)
    print(f"Similarity with context '{context}': {similarity}")

Fuzzy Similarity Score: 38
Similarity with context 'The First AFL–NFL World Championship Game was on January 15, 1967.': 59
Similarity with context 'Super Bowl I was played in 1967.': 57
Similarity with context 'This is completely unrelated.': 38


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk  # Make sure you have NLTK installed
nltk.download('punkt', quiet=True) # Download punkt tokenizer if you haven't already


def calculate_tfidf_cosine_similarity(response, context):
    """Calculates TF-IDF cosine similarity between response and context."""

    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([response, context])  # Fit and transform both strings

    similarity = cosine_similarity(vectors[0], vectors[1])[0][0] # cosine_similarity returns a 2d array
    return similarity


def calculate_bleu_score(response, context):
    """Calculates BLEU score between response and context."""

    reference = context.split()  # Split the context into words
    candidate = response.split()  # Split the response into words

    # Use a smoothing function to handle cases where n-grams are not present
    smoothing = SmoothingFunction().method4  # Or another smoothing method

    bleu = sentence_bleu([reference], candidate, smoothing_function=smoothing)
    return bleu



# Example usage:
user_input = "When was the first super bowl?"
response = "The first superbowl was held on Jan 15, 1967."
retrieved_context = "The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles."

tfidf_similarity = calculate_tfidf_cosine_similarity(response, retrieved_context)
bleu_score = calculate_bleu_score(response, retrieved_context)

print(f"TF-IDF Cosine Similarity: {tfidf_similarity:.2f}")
print(f"BLEU Score: {bleu_score:.2f}")


# Example with multiple contexts:
contexts = [
    "The First AFL–NFL World Championship Game was on January 15, 1967.",
    "Super Bowl I was played in 1967.",
    "This is completely unrelated."
]

for context in contexts:
    tfidf = calculate_tfidf_cosine_similarity(response, context)
    bleu = calculate_bleu_score(response, context)
    print(f"TF-IDF with context '{context}': {tfidf:.2f}")
    print(f"BLEU with context '{context}': {bleu:.2f}")

TF-IDF Cosine Similarity: 0.27
BLEU Score: 0.01
TF-IDF with context 'The First AFL–NFL World Championship Game was on January 15, 1967.': 0.41
BLEU with context 'The First AFL–NFL World Championship Game was on January 15, 1967.': 0.06
TF-IDF with context 'Super Bowl I was played in 1967.': 0.16
BLEU with context 'Super Bowl I was played in 1967.': 0.03
TF-IDF with context 'This is completely unrelated.': 0.00
BLEU with context 'This is completely unrelated.': 0.00
